In [1]:
print("Hi")

Hi


In [1]:
import tensorflow as tf

In [2]:
import tensorflow as tf

In [3]:
print("Hi")

Hi


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

# -----------------------------
# 1) Load data
# -----------------------------
train_path = "Delhi_train.csv"
edges_path = "Delhi_edges.csv"

df = pd.read_csv(train_path, header=None, names=["user", "time", "opinion"])
df["user"] = df["user"].astype(int)
df["time"] = df["time"].astype(int)
df["opinion"] = df["opinion"].astype(float)

A = pd.read_csv(edges_path, header=None).values.astype(np.float32)

num_users = df["user"].max() + 1
T = df["time"].max() + 1

In [6]:
# If multiple tweets per (user,time), average them:
df = df.groupby(["user", "time"], as_index=False)["opinion"].mean()
df.iloc[100:130]

,user,time,opinion
100,12,3,-0.248332
101,12,4,-0.084017
102,12,5,0.060703
103,12,6,-0.001325
104,12,7,0.027887
105,12,8,0.118700
106,13,0,0.095879
107,13,2,0.206460
108,13,3,-0.043402
109,13,4,0.132383


In [7]:
# -----------------------------
# 2) Build opinion tensor with NaNs
# -----------------------------
opinions = np.full((T, num_users), np.nan, dtype=np.float32)
for row in df.itertuples(index=False):
    opinions[row.time, row.user] = row.opinion

observed = ~np.isnan(opinions)  # (T, N) boolean

In [9]:
observed.sum()


4470

In [10]:
# -----------------------------
# 3) Build features: last_value, mask, delta_t
# -----------------------------
last_val = np.zeros((T, num_users), dtype=np.float32)
mask = np.zeros((T, num_users), dtype=np.float32)
delta = np.zeros((T, num_users), dtype=np.float32)

In [12]:
len(last_val)

9